In [2]:
import numpy as np
import pandas as pd
import yfinance as yf
from typing import Optional, Dict


def _safe_column(df: pd.DataFrame, col: str) -> np.ndarray:
    """
    Return the column as a numpy array if it exists, otherwise return an array of NaNs.
    """
    if col in df.columns:
        return df[col].to_numpy()
    else:
        return np.full(len(df), np.nan, dtype=float)


def _select_expiration(
    ticker_obj: yf.Ticker, expiration: Optional[str], interactive: bool
) -> str:
    """
    Resolve which expiration to use. If expiration is provided and valid, use it.
    If None and interactive=True, prompt the user. Otherwise pick the nearest upcoming.
    """
    available = ticker_obj.options
    if not available:
        raise ValueError(f"No options available for ticker '{ticker_obj.ticker}'")

    if expiration is None:
        if interactive:
            print(f"Available expirations for {ticker_obj.ticker}:")
            for idx, exp in enumerate(available):
                print(f"  [{idx}] {exp}")
            choice = input("Enter index or expiration date (leave empty for nearest): ").strip()
            if choice == "":
                return available[0]
            if choice.isdigit():
                i = int(choice)
                if i < 0 or i >= len(available):
                    raise ValueError(f"Index {i} out of range")
                return available[i]
            if choice in available:
                return choice
            raise ValueError(f"Expiration '{choice}' not in available list")
        else:
            return available[0]
    else:
        if expiration not in available:
            raise ValueError(f"Expiration '{expiration}' not in available expirations: {available}")
        return expiration


def options_bid_ask(
    firm: str,
    expiration: Optional[str] = None,
    interactive: bool = False,
) -> Dict[str, np.ndarray]:
    """
    Download option bid/ask/lastPrice data for calls and puts for a given ticker and expiration.

    Parameters:
        firm: Ticker symbol, e.g. "AAPL"
        expiration: Expiration date string like "2025-08-15". If None:
            - if interactive=True, user is prompted to choose;
            - else the nearest upcoming expiration is used.
        interactive: Whether to prompt the user to select expiration when not provided.

    Returns:
        Dictionary containing:
            stock_value: latest close price of the underlying
            expiration: the expiration actually used
            call_strikes, call_bid, call_ask, call_last_price
            put_strikes, put_bid, put_ask, put_last_price
    """
    # Initialize ticker object
    stock = yf.Ticker(firm)

    # Get latest close price of the underlying
    hist = stock.history(period="1d")
    if hist.empty:
        raise ValueError(f"Could not fetch price history for '{firm}'")
    stock_value = hist["Close"].iloc[-1]

    # Determine expiration to use
    chosen_exp = _select_expiration(stock, expiration, interactive)

    # Fetch option chain for the chosen expiration
    chain = stock.option_chain(chosen_exp)
    calls = chain.calls
    puts = chain.puts

    # Build result dictionary with separate vectors
    result = {
        "stock_value": stock_value,
        "expiration": chosen_exp,
        "call_strikes": calls["strike"].to_numpy(),
        "call_bid": _safe_column(calls, "bid"),
        "call_ask": _safe_column(calls, "ask"),
        "call_last_price": _safe_column(calls, "lastPrice"),
        "put_strikes": puts["strike"].to_numpy(),
        "put_bid": _safe_column(puts, "bid"),
        "put_ask": _safe_column(puts, "ask"),
        "put_last_price": _safe_column(puts, "lastPrice"),
    }
    return result


# Example usage
if __name__ == "__main__":
    # Example: user can choose expiration interactively
    ticker = "AAPL"
    data = options_bid_ask(ticker, expiration=None, interactive=True)

    # Print underlying and chosen expiration
    print(f"Underlying last close: {data['stock_value']}")
    print(f"Using expiration: {data['expiration']}")

    # Display sample call data
    print("\nSample Call (strike, bid, ask, lastPrice):")
    for strike, bid, ask, last in zip(
        data["call_strikes"][:5],
        data["call_bid"][:5],
        data["call_ask"][:5],
        data["call_last_price"][:5],
    ):
        print(f"  Strike: {strike}, Bid: {bid}, Ask: {ask}, Last: {last}")

    # Display sample put data
    print("\nSample Put (strike, bid, ask, lastPrice):")
    for strike, bid, ask, last in zip(
        data["put_strikes"][:5],
        data["put_bid"][:5],
        data["put_ask"][:5],
        data["put_last_price"][:5],
    ):
        print(f"  Strike: {strike}, Bid: {bid}, Ask: {ask}, Last: {last}")


Available expirations for AAPL:
  [0] 2025-08-08
  [1] 2025-08-15
  [2] 2025-08-22
  [3] 2025-08-29
  [4] 2025-09-05
  [5] 2025-09-19
  [6] 2025-10-17
  [7] 2025-11-21
  [8] 2025-12-19
  [9] 2026-01-16
  [10] 2026-02-20
  [11] 2026-03-20
  [12] 2026-05-15
  [13] 2026-06-18
  [14] 2026-09-18
  [15] 2026-12-18
  [16] 2027-01-15
  [17] 2027-06-17
  [18] 2027-12-17
Enter index or expiration date (leave empty for nearest): 2
Underlying last close: 202.3800048828125
Using expiration: 2025-08-22

Sample Call (strike, bid, ask, lastPrice):
  Strike: 110.0, Bid: 91.25, Ask: 93.35, Last: 97.65
  Strike: 120.0, Bid: 81.7, Ask: 83.65, Last: 94.76
  Strike: 130.0, Bid: 71.65, Ask: 73.8, Last: 83.23
  Strike: 135.0, Bid: 66.3, Ask: 68.2, Last: 71.97
  Strike: 140.0, Bid: 61.15, Ask: 63.5, Last: 67.02

Sample Put (strike, bid, ask, lastPrice):
  Strike: 110.0, Bid: 0.0, Ask: 0.03, Last: 0.01
  Strike: 120.0, Bid: 0.0, Ask: 0.03, Last: 0.02
  Strike: 125.0, Bid: 0.0, Ask: 0.11, Last: 0.03
  Strike: 13